# General data reader for AQUA 
## Streaming mode

The reader includes the ability to simulate a data streaming, that allows to retrieve chunks of data of specific time length.

In [1]:
from aqua import Reader

Let's load some IFS data. We first instantiate a `Reader` object specifying the type of data which we want to read from the catalogue. 
By default, the streaming mode is deactivated (`streaming = False`) and the `retrieve` method will return all the requested data. If you want to activate the streaming mode you should specify the argument `streaming=True`.

There are also three optional parameters: `startdate`, `enddate` and `aggregation`: 

`startdate` : is a string representing the starting date for streaming the data (e.g. '2020-02-25'). If it is not specified, the function will use the first date in the dataset.

`enddate`: is a string representing the ending date for streaming the data (e.g. '2021-01-01').  If it is not specified, the function will use the last date in the dataset.

`aggregation` (str): the streaming frequency.
 Available frequencies: 
- 1S, 5S etc ('S' stands for 'steps', if you want to keep the original data frequency. On hourly data, '5S' will return chunks of 5 hours each.)
  If no 'aggregation' is specified, '1S' will be the default streaming frequency.

- Panda's style: 1M, 7D etc. (keep in mind that the behavior of resampling in Pandas can be a bit counterintuitive when it comes to months (years) due to the variation in the length of months(years). It's safer to ask for multiple chunks of just hours or days.) 

- 'hourly', 'daily', 'weekly', 'monthly', 'yearly'.

- 'hour', 'day', 'pentad', 'week', 'month', 'year', 'decade'  (and their plural).
                                                                    
Keep in mind that if you want to stream chunks of a certain number of days, months etc. you must use the Panda's notation (e.g if you want to stream chunks of three months you should write `aggregation = '3M'`, `aggregation = '3months'` is not accepted).

Let's say we want to stream the data every three days. We need to call:

In [2]:
reader = Reader(model='OSI-SAF',exp='osi-450',source='sh-daily', streaming=True, aggregation = '3D')

/LUMI_TYKKY_ZOMGa5S/miniconda/envs/env1/lib/python3.11/site-packages/intake_xarray/base.py:21: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  'dims': dict(self._ds.dims),


Then the actual data are read with the `retrieve` method. By default, the streaming mode is deactivated (`streaming = False`) and the `retrieve` method will return all the requested data. If you want to activate the streaming mode you should specify the argument `streaming=True`.

There are also three optional parameters: `stream_step`, `stream_unit` and `stream_startdate`: 

`stream_unit` is a string representing the unit of time to stream the data by (e.g. 'hours', 'days', 'months') (default is None)

`stream_step` is an integer representing the number of time steps to stream the data by.  Default = 1

`stream_startdate` is a string representing the starting date for streaming the data (e.g. '2020-02-25'). If it is not specified, the function will use the first date in the dataset.

Let's say we want to stream the data every three days. We need to call:

In [3]:
data = reader.retrieve()
data

2024-02-12 19:55:10 :: Reader :: ERROR    -> Variable time_bnds has no units!
2024-02-12 19:55:10 :: Reader :: ERROR    -> Variable Lambert_Azimuthal_Grid has no units!
2024-02-12 19:55:10 :: Reader :: ERROR    -> Variable status_flag has no units!


<xarray.Dataset>
Dimensions:                   (time: 2, bnds: 2, yc: 432, xc: 432)
Coordinates:
  * time                      (time) datetime64[ns] 1979-01-02T12:00:00 1979-...
    lon                       (yc, xc) float32 dask.array<chunksize=(432, 432), meta=np.ndarray>
    lat                       (yc, xc) float32 dask.array<chunksize=(432, 432), meta=np.ndarray>
  * xc                        (xc) float64 -5.388e+03 -5.362e+03 ... 5.388e+03
  * yc                        (yc) float64 5.388e+03 5.362e+03 ... -5.388e+03
Dimensions without coordinates: bnds
Data variables:
    time_bnds                 (time, bnds) datetime64[ns] dask.array<chunksize=(1, 2), meta=np.ndarray>
    Lambert_Azimuthal_Grid    (time) int32 1 1
    avg_siconc                (time, yc, xc) float64 dask.array<chunksize=(1, 432, 432), meta=np.ndarray>
    raw_ice_conc_values       (time, yc, xc) float64 dask.array<chunksize=(1, 432, 432), meta=np.ndarray>
    total_standard_error      (time, yc, xc) float64 dask.array<chunksize=(1, 432, 432), meta=np.ndarray>
    smearing_standard_error   (time, yc, xc) float64 dask.array<chunksize=(1, 432, 432), meta=np.ndarray>
    algorithm_standard_error  (time, yc, xc) float64 dask.array<chunksize=(1, 432, 432), meta=np.ndarray>
    status_flag               (time, yc, xc) float32 dask.array<chunksize=(1, 432, 432), meta=np.ndarray>
Attributes: (12/42)
    CDI:                       Climate Data Interface version 2.0.6 (https://...
    source:                    FCDR of SMMR / SSMI / SSMIS Brightness Tempera...
    institution:               OSISAF - EUMETSAT
    Conventions:               CF-1.6 ACDD-1.3
    title:                     Sea Ice Concentration Climate Data Record from...
    summary:                   This climate data record of sea ice concentrat...
    ...                        ...
    standard_name_vocabulary:  CF Standard Name Table (v36, 21 September 2016)
    id:                        10.15770/EUM_SAF_OSI_0008
    product_version:           2.0
    history:                   Fri May 05 09:16:25 2023: cdo cat /work/bb1153...
    geospatial_bounds_crs:     EPSG:6932
    CDO:                       Climate Data Operators version 2.0.6 (https://...

We correctly retrieved data from 2020-01-20 to 2020-01-23, keeping the original hourly temporal resolution. 

If `retrieve` is called a second time, it will return the subsequent chunk of 3 days data. This is because the method keeps track of the state of the streaming process through the use of internal attributes.

In [4]:
data = reader.retrieve()
data

2024-02-12 19:55:38 :: Reader :: ERROR    -> Variable time_bnds has no units!
2024-02-12 19:55:38 :: Reader :: ERROR    -> Variable Lambert_Azimuthal_Grid has no units!
2024-02-12 19:55:38 :: Reader :: ERROR    -> Variable status_flag has no units!


<xarray.Dataset>
Dimensions:                   (time: 1, bnds: 2, yc: 432, xc: 432)
Coordinates:
  * time                      (time) datetime64[ns] 1979-01-06T12:00:00
    lon                       (yc, xc) float32 dask.array<chunksize=(432, 432), meta=np.ndarray>
    lat                       (yc, xc) float32 dask.array<chunksize=(432, 432), meta=np.ndarray>
  * xc                        (xc) float64 -5.388e+03 -5.362e+03 ... 5.388e+03
  * yc                        (yc) float64 5.388e+03 5.362e+03 ... -5.388e+03
Dimensions without coordinates: bnds
Data variables:
    time_bnds                 (time, bnds) datetime64[ns] dask.array<chunksize=(1, 2), meta=np.ndarray>
    Lambert_Azimuthal_Grid    (time) int32 1
    avg_siconc                (time, yc, xc) float64 dask.array<chunksize=(1, 432, 432), meta=np.ndarray>
    raw_ice_conc_values       (time, yc, xc) float64 dask.array<chunksize=(1, 432, 432), meta=np.ndarray>
    total_standard_error      (time, yc, xc) float64 dask.array<chunksize=(1, 432, 432), meta=np.ndarray>
    smearing_standard_error   (time, yc, xc) float64 dask.array<chunksize=(1, 432, 432), meta=np.ndarray>
    algorithm_standard_error  (time, yc, xc) float64 dask.array<chunksize=(1, 432, 432), meta=np.ndarray>
    status_flag               (time, yc, xc) float32 dask.array<chunksize=(1, 432, 432), meta=np.ndarray>
Attributes: (12/42)
    CDI:                       Climate Data Interface version 2.0.6 (https://...
    source:                    FCDR of SMMR / SSMI / SSMIS Brightness Tempera...
    institution:               OSISAF - EUMETSAT
    Conventions:               CF-1.6 ACDD-1.3
    title:                     Sea Ice Concentration Climate Data Record from...
    summary:                   This climate data record of sea ice concentrat...
    ...                        ...
    standard_name_vocabulary:  CF Standard Name Table (v36, 21 September 2016)
    id:                        10.15770/EUM_SAF_OSI_0008
    product_version:           2.0
    history:                   Fri May 05 09:16:25 2023: cdo cat /work/bb1153...
    geospatial_bounds_crs:     EPSG:6932
    CDO:                       Climate Data Operators version 2.0.6 (https://...

By calling `retrieve` a second time, we correctly retrieved data from 2020-01-23 to 2020-01-26.

If the retrieve method in streaming mode is called multiple times, it will return the data in chunks until all of the data has been streamed. The function will automatically determine the appropriate start and end points for each chunk based on the internal state of the streaming process.

If we want to reset the state of the streaming process we can call the `reset_stream` method. This means that if `retrieve` in streaming mode is called again after calling `reset_stream`, it will start streaming the input data from the beginning.

In [5]:
reader.reset_stream()
data = reader.retrieve()
data

2024-02-12 19:55:44 :: Reader :: ERROR    -> Variable time_bnds has no units!
2024-02-12 19:55:44 :: Reader :: ERROR    -> Variable Lambert_Azimuthal_Grid has no units!
2024-02-12 19:55:44 :: Reader :: ERROR    -> Variable status_flag has no units!


<xarray.Dataset>
Dimensions:                   (time: 2, bnds: 2, yc: 432, xc: 432)
Coordinates:
  * time                      (time) datetime64[ns] 1979-01-02T12:00:00 1979-...
    lon                       (yc, xc) float32 dask.array<chunksize=(432, 432), meta=np.ndarray>
    lat                       (yc, xc) float32 dask.array<chunksize=(432, 432), meta=np.ndarray>
  * xc                        (xc) float64 -5.388e+03 -5.362e+03 ... 5.388e+03
  * yc                        (yc) float64 5.388e+03 5.362e+03 ... -5.388e+03
Dimensions without coordinates: bnds
Data variables:
    time_bnds                 (time, bnds) datetime64[ns] dask.array<chunksize=(1, 2), meta=np.ndarray>
    Lambert_Azimuthal_Grid    (time) int32 1 1
    avg_siconc                (time, yc, xc) float64 dask.array<chunksize=(1, 432, 432), meta=np.ndarray>
    raw_ice_conc_values       (time, yc, xc) float64 dask.array<chunksize=(1, 432, 432), meta=np.ndarray>
    total_standard_error      (time, yc, xc) float64 dask.array<chunksize=(1, 432, 432), meta=np.ndarray>
    smearing_standard_error   (time, yc, xc) float64 dask.array<chunksize=(1, 432, 432), meta=np.ndarray>
    algorithm_standard_error  (time, yc, xc) float64 dask.array<chunksize=(1, 432, 432), meta=np.ndarray>
    status_flag               (time, yc, xc) float32 dask.array<chunksize=(1, 432, 432), meta=np.ndarray>
Attributes: (12/42)
    CDI:                       Climate Data Interface version 2.0.6 (https://...
    source:                    FCDR of SMMR / SSMI / SSMIS Brightness Tempera...
    institution:               OSISAF - EUMETSAT
    Conventions:               CF-1.6 ACDD-1.3
    title:                     Sea Ice Concentration Climate Data Record from...
    summary:                   This climate data record of sea ice concentrat...
    ...                        ...
    standard_name_vocabulary:  CF Standard Name Table (v36, 21 September 2016)
    id:                        10.15770/EUM_SAF_OSI_0008
    product_version:           2.0
    history:                   Fri May 05 09:16:25 2023: cdo cat /work/bb1153...
    geospatial_bounds_crs:     EPSG:6932
    CDO:                       Climate Data Operators version 2.0.6 (https://...

We got back to the first three days, 2020-01-20 to 2020-01-23.

By default, the data stream will start from the first date of the input file.
If we want it to start from a specific date, we can use the parameter `startdate`.

In [7]:
reader = Reader(model='OSI-SAF',exp='osi-450',source='sh-daily', streaming=True, aggregation = '3D', startdate = '2010-05-01')
data = reader.retrieve()
data

/LUMI_TYKKY_ZOMGa5S/miniconda/envs/env1/lib/python3.11/site-packages/intake_xarray/base.py:21: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  'dims': dict(self._ds.dims),
2024-02-12 19:57:09 :: Reader :: ERROR    -> Variable time_bnds has no units!
2024-02-12 19:57:09 :: Reader :: ERROR    -> Variable Lambert_Azimuthal_Grid has no units!
2024-02-12 19:57:09 :: Reader :: ERROR    -> Variable status_flag has no units!


<xarray.Dataset>
Dimensions:                   (time: 3, bnds: 2, yc: 432, xc: 432)
Coordinates:
  * time                      (time) datetime64[ns] 2010-05-01T12:00:00 ... 2...
    lon                       (yc, xc) float32 dask.array<chunksize=(432, 432), meta=np.ndarray>
    lat                       (yc, xc) float32 dask.array<chunksize=(432, 432), meta=np.ndarray>
  * xc                        (xc) float64 -5.388e+03 -5.362e+03 ... 5.388e+03
  * yc                        (yc) float64 5.388e+03 5.362e+03 ... -5.388e+03
Dimensions without coordinates: bnds
Data variables:
    time_bnds                 (time, bnds) datetime64[ns] dask.array<chunksize=(1, 2), meta=np.ndarray>
    Lambert_Azimuthal_Grid    (time) int32 1 1 1
    avg_siconc                (time, yc, xc) float64 dask.array<chunksize=(1, 432, 432), meta=np.ndarray>
    raw_ice_conc_values       (time, yc, xc) float64 dask.array<chunksize=(1, 432, 432), meta=np.ndarray>
    total_standard_error      (time, yc, xc) float64 dask.array<chunksize=(1, 432, 432), meta=np.ndarray>
    smearing_standard_error   (time, yc, xc) float64 dask.array<chunksize=(1, 432, 432), meta=np.ndarray>
    algorithm_standard_error  (time, yc, xc) float64 dask.array<chunksize=(1, 432, 432), meta=np.ndarray>
    status_flag               (time, yc, xc) float32 dask.array<chunksize=(1, 432, 432), meta=np.ndarray>
Attributes: (12/42)
    CDI:                       Climate Data Interface version 2.0.6 (https://...
    source:                    FCDR of SMMR / SSMI / SSMIS Brightness Tempera...
    institution:               OSISAF - EUMETSAT
    Conventions:               CF-1.6 ACDD-1.3
    title:                     Sea Ice Concentration Climate Data Record from...
    summary:                   This climate data record of sea ice concentrat...
    ...                        ...
    standard_name_vocabulary:  CF Standard Name Table (v36, 21 September 2016)
    id:                        10.15770/EUM_SAF_OSI_0008
    product_version:           2.0
    history:                   Fri May 05 09:16:25 2023: cdo cat /work/bb1153...
    geospatial_bounds_crs:     EPSG:6932
    CDO:                       Climate Data Operators version 2.0.6 (https://...

Similarly, if we want to end the streaming at a specific date, before the natural end of the dataset, we can use the parameter `enddate`.

In [9]:
reader = Reader(model='OSI-SAF',exp='osi-450',source='sh-daily', streaming=True, aggregation = '3D', startdate = '2010-05-01', enddate = '2010-05-05')
data = reader.retrieve()
data

/LUMI_TYKKY_ZOMGa5S/miniconda/envs/env1/lib/python3.11/site-packages/intake_xarray/base.py:21: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  'dims': dict(self._ds.dims),
2024-02-12 20:03:14 :: Reader :: ERROR    -> Variable time_bnds has no units!
2024-02-12 20:03:14 :: Reader :: ERROR    -> Variable Lambert_Azimuthal_Grid has no units!
2024-02-12 20:03:14 :: Reader :: ERROR    -> Variable status_flag has no units!


<xarray.Dataset>
Dimensions:                   (time: 3, bnds: 2, yc: 432, xc: 432)
Coordinates:
  * time                      (time) datetime64[ns] 2010-05-01T12:00:00 ... 2...
    lon                       (yc, xc) float32 dask.array<chunksize=(432, 432), meta=np.ndarray>
    lat                       (yc, xc) float32 dask.array<chunksize=(432, 432), meta=np.ndarray>
  * xc                        (xc) float64 -5.388e+03 -5.362e+03 ... 5.388e+03
  * yc                        (yc) float64 5.388e+03 5.362e+03 ... -5.388e+03
Dimensions without coordinates: bnds
Data variables:
    time_bnds                 (time, bnds) datetime64[ns] dask.array<chunksize=(1, 2), meta=np.ndarray>
    Lambert_Azimuthal_Grid    (time) int32 1 1 1
    avg_siconc                (time, yc, xc) float64 dask.array<chunksize=(1, 432, 432), meta=np.ndarray>
    raw_ice_conc_values       (time, yc, xc) float64 dask.array<chunksize=(1, 432, 432), meta=np.ndarray>
    total_standard_error      (time, yc, xc) float64 dask.array<chunksize=(1, 432, 432), meta=np.ndarray>
    smearing_standard_error   (time, yc, xc) float64 dask.array<chunksize=(1, 432, 432), meta=np.ndarray>
    algorithm_standard_error  (time, yc, xc) float64 dask.array<chunksize=(1, 432, 432), meta=np.ndarray>
    status_flag               (time, yc, xc) float32 dask.array<chunksize=(1, 432, 432), meta=np.ndarray>
Attributes: (12/42)
    CDI:                       Climate Data Interface version 2.0.6 (https://...
    source:                    FCDR of SMMR / SSMI / SSMIS Brightness Tempera...
    institution:               OSISAF - EUMETSAT
    Conventions:               CF-1.6 ACDD-1.3
    title:                     Sea Ice Concentration Climate Data Record from...
    summary:                   This climate data record of sea ice concentrat...
    ...                        ...
    standard_name_vocabulary:  CF Standard Name Table (v36, 21 September 2016)
    id:                        10.15770/EUM_SAF_OSI_0008
    product_version:           2.0
    history:                   Fri May 05 09:16:25 2023: cdo cat /work/bb1153...
    geospatial_bounds_crs:     EPSG:6932
    CDO:                       Climate Data Operators version 2.0.6 (https://...

In [10]:
data = reader.retrieve()
data

2024-02-12 20:03:47 :: Reader :: ERROR    -> Variable time_bnds has no units!
2024-02-12 20:03:47 :: Reader :: ERROR    -> Variable Lambert_Azimuthal_Grid has no units!
2024-02-12 20:03:47 :: Reader :: ERROR    -> Variable status_flag has no units!


<xarray.Dataset>
Dimensions:                   (time: 2, bnds: 2, yc: 432, xc: 432)
Coordinates:
  * time                      (time) datetime64[ns] 2010-05-04T12:00:00 2010-...
    lon                       (yc, xc) float32 dask.array<chunksize=(432, 432), meta=np.ndarray>
    lat                       (yc, xc) float32 dask.array<chunksize=(432, 432), meta=np.ndarray>
  * xc                        (xc) float64 -5.388e+03 -5.362e+03 ... 5.388e+03
  * yc                        (yc) float64 5.388e+03 5.362e+03 ... -5.388e+03
Dimensions without coordinates: bnds
Data variables:
    time_bnds                 (time, bnds) datetime64[ns] dask.array<chunksize=(1, 2), meta=np.ndarray>
    Lambert_Azimuthal_Grid    (time) int32 1 1
    avg_siconc                (time, yc, xc) float64 dask.array<chunksize=(1, 432, 432), meta=np.ndarray>
    raw_ice_conc_values       (time, yc, xc) float64 dask.array<chunksize=(1, 432, 432), meta=np.ndarray>
    total_standard_error      (time, yc, xc) float64 dask.array<chunksize=(1, 432, 432), meta=np.ndarray>
    smearing_standard_error   (time, yc, xc) float64 dask.array<chunksize=(1, 432, 432), meta=np.ndarray>
    algorithm_standard_error  (time, yc, xc) float64 dask.array<chunksize=(1, 432, 432), meta=np.ndarray>
    status_flag               (time, yc, xc) float32 dask.array<chunksize=(1, 432, 432), meta=np.ndarray>
Attributes: (12/42)
    CDI:                       Climate Data Interface version 2.0.6 (https://...
    source:                    FCDR of SMMR / SSMI / SSMIS Brightness Tempera...
    institution:               OSISAF - EUMETSAT
    Conventions:               CF-1.6 ACDD-1.3
    title:                     Sea Ice Concentration Climate Data Record from...
    summary:                   This climate data record of sea ice concentrat...
    ...                        ...
    standard_name_vocabulary:  CF Standard Name Table (v36, 21 September 2016)
    id:                        10.15770/EUM_SAF_OSI_0008
    product_version:           2.0
    history:                   Fri May 05 09:16:25 2023: cdo cat /work/bb1153...
    geospatial_bounds_crs:     EPSG:6932
    CDO:                       Climate Data Operators version 2.0.6 (https://...

As expected, the second chunk is shorter than the requested 3 days as we have reached the end date.

As mentioned above, if we don't specify the `aggregation` parameter, the data is streamed keeping the the original time resolution of input data
(e.g. hour by hour for hourly data)
If we want chunks of a certain number of steps, we need to specify it. (e.g. aggregations = '6S')


In [11]:
reader = Reader(model='OSI-SAF',exp='osi-450',source='sh-daily', streaming=True, aggregation = '6S')
data = reader.retrieve()
data

/LUMI_TYKKY_ZOMGa5S/miniconda/envs/env1/lib/python3.11/site-packages/intake_xarray/base.py:21: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  'dims': dict(self._ds.dims),
2024-02-12 20:04:29 :: Reader :: ERROR    -> Variable time_bnds has no units!
2024-02-12 20:04:29 :: Reader :: ERROR    -> Variable Lambert_Azimuthal_Grid has no units!
2024-02-12 20:04:29 :: Reader :: ERROR    -> Variable status_flag has no units!


<xarray.Dataset>
Dimensions:                   (time: 6, bnds: 2, yc: 432, xc: 432)
Coordinates:
  * time                      (time) datetime64[ns] 1979-01-02T12:00:00 ... 1...
    lon                       (yc, xc) float32 dask.array<chunksize=(432, 432), meta=np.ndarray>
    lat                       (yc, xc) float32 dask.array<chunksize=(432, 432), meta=np.ndarray>
  * xc                        (xc) float64 -5.388e+03 -5.362e+03 ... 5.388e+03
  * yc                        (yc) float64 5.388e+03 5.362e+03 ... -5.388e+03
Dimensions without coordinates: bnds
Data variables:
    time_bnds                 (time, bnds) datetime64[ns] dask.array<chunksize=(1, 2), meta=np.ndarray>
    Lambert_Azimuthal_Grid    (time) int32 1 1 1 1 1 1
    avg_siconc                (time, yc, xc) float64 dask.array<chunksize=(1, 432, 432), meta=np.ndarray>
    raw_ice_conc_values       (time, yc, xc) float64 dask.array<chunksize=(1, 432, 432), meta=np.ndarray>
    total_standard_error      (time, yc, xc) float64 dask.array<chunksize=(1, 432, 432), meta=np.ndarray>
    smearing_standard_error   (time, yc, xc) float64 dask.array<chunksize=(1, 432, 432), meta=np.ndarray>
    algorithm_standard_error  (time, yc, xc) float64 dask.array<chunksize=(1, 432, 432), meta=np.ndarray>
    status_flag               (time, yc, xc) float32 dask.array<chunksize=(1, 432, 432), meta=np.ndarray>
Attributes: (12/42)
    CDI:                       Climate Data Interface version 2.0.6 (https://...
    source:                    FCDR of SMMR / SSMI / SSMIS Brightness Tempera...
    institution:               OSISAF - EUMETSAT
    Conventions:               CF-1.6 ACDD-1.3
    title:                     Sea Ice Concentration Climate Data Record from...
    summary:                   This climate data record of sea ice concentrat...
    ...                        ...
    standard_name_vocabulary:  CF Standard Name Table (v36, 21 September 2016)
    id:                        10.15770/EUM_SAF_OSI_0008
    product_version:           2.0
    history:                   Fri May 05 09:16:25 2023: cdo cat /work/bb1153...
    geospatial_bounds_crs:     EPSG:6932
    CDO:                       Climate Data Operators version 2.0.6 (https://...

As requested, we got a six-hours chunk.

Another possibility to deal with data streaming is to call the `generator` method of the `Streaming` class.
This can be done in the Reader initialization through the argument `stream_generator = True`.

In [1]:
from aqua import Reader

In [3]:
reader = Reader(model='OSI-SAF',exp='osi-450',source='sh-daily', stream_generator = 'True', aggregation = 'monthly', startdate = '2012-05-01')
data_gen = reader.retrieve()
data_gen

/LUMI_TYKKY_ZOMGa5S/miniconda/envs/env1/lib/python3.11/site-packages/intake_xarray/base.py:21: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  'dims': dict(self._ds.dims),
2024-02-12 20:21:42 :: Reader :: ERROR    -> Variable time_bnds has no units!
2024-02-12 20:21:42 :: Reader :: ERROR    -> Variable Lambert_Azimuthal_Grid has no units!
2024-02-12 20:21:42 :: Reader :: ERROR    -> Variable status_flag has no units!


<generator object Streaming.generator at 0x7f5728d58dc0>

`data_gen` is now a generator object that yields the requested one-month-long chunks of data. We can do operations with them by iterating on the generator object:

In [4]:
for data in data_gen:
    print(f"start_date: {data.time[0].values} stop_date: {data.time[-1].values}")

start_date: 2012-05-01T12:00:00.000000000 stop_date: 2012-05-31T12:00:00.000000000
start_date: 2012-06-01T12:00:00.000000000 stop_date: 2012-06-30T12:00:00.000000000
start_date: 2012-07-01T12:00:00.000000000 stop_date: 2012-07-31T12:00:00.000000000
start_date: 2012-08-01T12:00:00.000000000 stop_date: 2012-08-31T12:00:00.000000000
start_date: 2012-09-01T12:00:00.000000000 stop_date: 2012-09-30T12:00:00.000000000
start_date: 2012-10-01T12:00:00.000000000 stop_date: 2012-10-31T12:00:00.000000000
start_date: 2012-11-01T12:00:00.000000000 stop_date: 2012-11-30T12:00:00.000000000
start_date: 2012-12-01T12:00:00.000000000 stop_date: 2012-12-31T12:00:00.000000000
start_date: 2013-01-01T12:00:00.000000000 stop_date: 2013-01-31T12:00:00.000000000
start_date: 2013-02-01T12:00:00.000000000 stop_date: 2013-02-28T12:00:00.000000000
start_date: 2013-03-01T12:00:00.000000000 stop_date: 2013-03-31T12:00:00.000000000
start_date: 2013-04-01T12:00:00.000000000 stop_date: 2013-04-30T12:00:00.000000000
star

An useful feature provided by the 'stream_generator' option is the possibility to have generators with different `startdate` and `enddate`.

In [5]:
reader2 = Reader(model='OSI-SAF',exp='osi-450',source='sh-daily', stream_generator = 'True', startdate = '2013-11-01', aggregation = 'monthly')
data_gen2 = reader2.retrieve()
for data in data_gen2:
    print(f"start_date: {data.time[0].values} stop_date: {data.time[-1].values}")

/LUMI_TYKKY_ZOMGa5S/miniconda/envs/env1/lib/python3.11/site-packages/intake_xarray/base.py:21: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  'dims': dict(self._ds.dims),
2024-02-12 20:29:44 :: Reader :: ERROR    -> Variable time_bnds has no units!
2024-02-12 20:29:44 :: Reader :: ERROR    -> Variable Lambert_Azimuthal_Grid has no units!
2024-02-12 20:29:44 :: Reader :: ERROR    -> Variable status_flag has no units!


start_date: 2013-11-01T12:00:00.000000000 stop_date: 2013-11-30T12:00:00.000000000
start_date: 2013-12-01T12:00:00.000000000 stop_date: 2013-12-31T12:00:00.000000000
start_date: 2014-01-01T12:00:00.000000000 stop_date: 2014-01-31T12:00:00.000000000
start_date: 2014-02-01T12:00:00.000000000 stop_date: 2014-02-28T12:00:00.000000000
start_date: 2014-03-01T12:00:00.000000000 stop_date: 2014-03-31T12:00:00.000000000
start_date: 2014-04-01T12:00:00.000000000 stop_date: 2014-04-30T12:00:00.000000000
start_date: 2014-05-01T12:00:00.000000000 stop_date: 2014-05-31T12:00:00.000000000
start_date: 2014-06-01T12:00:00.000000000 stop_date: 2014-06-30T12:00:00.000000000
start_date: 2014-07-01T12:00:00.000000000 stop_date: 2014-07-31T12:00:00.000000000
start_date: 2014-08-01T12:00:00.000000000 stop_date: 2014-08-31T12:00:00.000000000
start_date: 2014-09-01T12:00:00.000000000 stop_date: 2014-09-30T12:00:00.000000000
start_date: 2014-10-01T12:00:00.000000000 stop_date: 2014-10-31T12:00:00.000000000
star

Another possibility is to use directly the `stream_data` method of the Reader class on a given xarray.Dataset, for example on a subset of the input data:

In [7]:
reader = Reader(model='OSI-SAF',exp='osi-450',source='sh-daily')
data = reader.retrieve()
new_data = reader.stream(data["avg_siconc"], aggregation='3D')
new_data

/LUMI_TYKKY_ZOMGa5S/miniconda/envs/env1/lib/python3.11/site-packages/intake_xarray/base.py:21: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  'dims': dict(self._ds.dims),
2024-02-12 20:30:48 :: Reader :: ERROR    -> Variable time_bnds has no units!
2024-02-12 20:30:48 :: Reader :: ERROR    -> Variable Lambert_Azimuthal_Grid has no units!
2024-02-12 20:30:48 :: Reader :: ERROR    -> Variable status_flag has no units!


<xarray.DataArray 'avg_siconc' (time: 2, yc: 432, xc: 432)>
dask.array<getitem, shape=(2, 432, 432), dtype=float64, chunksize=(1, 432, 432), chunktype=numpy.ndarray>
Coordinates:
  * time     (time) datetime64[ns] 1979-01-02T12:00:00 1979-01-04T12:00:00
    lon      (yc, xc) float32 dask.array<chunksize=(432, 432), meta=np.ndarray>
    lat      (yc, xc) float32 dask.array<chunksize=(432, 432), meta=np.ndarray>
  * xc       (xc) float64 -5.388e+03 -5.362e+03 ... 5.362e+03 5.388e+03
  * yc       (yc) float64 5.388e+03 5.362e+03 ... -5.362e+03 -5.388e+03
Attributes: (12/14)
    standard_name:        sea_ice_area_fraction
    long_name:            Time-mean sea ice area fraction
    units:                Fraction
    grid_mapping:         Lambert_Azimuthal_Grid
    ancillary_variables:  total_standard_error status_flag
    comment:              this field is the primary sea ice concentration est...
    ...                   ...
    cfVarName:            avg_siconc
    shortName:            avg_siconc
    factor:               0.01
    offset:               0
    src_units:            %
    units_fixed:          1